# Chinook Database

We will be working with chinook.db to answer business questions using SQL.

## Connecting to Database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Detail of Chinook Database

We will write query to see information on the tables and views in the database.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting New Albums to Purchase

Our client Chinook is considering adding three new albums to their list and has given us task to find number of tracks sold in USA by each genre. Our query will list genre by number of tracks sold in USA and percentage of total sold.

In [3]:
%%sql
WITH customer_usa_invoice AS (
                                SELECT 
                                        i.invoice_id 
                                FROM invoice i INNER JOIN (SELECT customer_id FROM customer WHERE country='USA') cu
                                ON cu.customer_id = i.customer_id
                             ),
customer_usa_tracks AS ( 
                        SELECT 
                                il.track_id
                        FROM invoice_line il INNER JOIN customer_usa_invoice cui
                        ON cui.invoice_id = il.invoice_id
                        ),

customer_usa_genre AS  (
                        SELECT 
                                t.genre_id, 
                                COUNT(t.track_id) number_of_track_sold
                                
                        FROM track t INNER JOIN customer_usa_tracks cut
                        ON cut.track_id = t.track_id
                        GROUP BY genre_id
                        )


SELECT 
        cug.genre_id, 
        g.name genre_name, number_of_track_sold, 
        number_of_track_sold / CAST((SELECT COUNT(track_id) from customer_usa_tracks) AS Float) percentage_sold
FROM customer_usa_genre cug INNER JOIN genre g
ON g.genre_id = cug.genre_id
ORDER BY number_of_track_sold DESC;                    

Done.


genre_id,genre_name,number_of_track_sold,percentage_sold
1,Rock,561,0.5337773549000951
4,Alternative & Punk,130,0.12369172216936251
3,Metal,124,0.11798287345385347
14,R&B/Soul,53,0.05042816365366318
6,Blues,36,0.03425309229305423
23,Alternative,35,0.03330161750713606
7,Latin,22,0.02093244529019981
9,Pop,22,0.02093244529019981
17,Hip Hop/Rap,20,0.019029495718363463
2,Jazz,14,0.013320647002854425


Above query showed rank of genre in tracks sold. We will suggest three artist from the list of four new artist based on genre ranking:

* Red Tone - Punk
* Slim Jim Bites - Blues
* Meteor and the Girls - Pop

## Sales Support Agent Performance

The Chinook has asked to analyze the purchases of customers belonging to each employee to check their performances.

We will write a query that finds total dollar amount of sales assigned to each sales support agent.

In [4]:
%%sql
WITH customer_total AS
                        (
                            SELECT i.customer_id, SUM(i.total) total_purchase, c.support_rep_id
                            FROM invoice i INNER JOIN customer c
                            ON c.customer_id = i.customer_id
                            GROUP BY i.customer_id
                        ),
    rep_total AS
                    ( 
                        SELECT support_rep_id, SUM(total_purchase) total_sales
                        FROM customer_total
                        GROUP BY support_rep_id
                    )
        
SELECT e.first_name||' '||e.last_name employee_name, e.hire_date, ROUND(rt.total_sales,2) total_sales
FROM employee e INNER JOIN rep_total rt 
ON rt.support_rep_id = e.employee_id
ORDER BY rt.total_sales DESC;

Done.


employee_name,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


Above query showed Jane had highest total_sales amount but we cannot conclude since she was hired earlier than other employees. 

# Analyzing Sales by Country

We will be analyzing the data for each country for following:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

Because there are number of countries with only one customer, we will group these customers as 'Other in our analysis.

In [5]:
%%sql
WITH customer_country AS (
                            SELECT 
                                    CASE
                                    WHEN (SELECT COUNT(*) 
                                        FROM customer
                                        WHERE country=c.country)=1 THEN 'Other'
                                        ELSE c.country
                                    END AS country,
                                    c.customer_id
                            FROM customer c
                            ), 
country_total AS (
                    SELECT cc.country, 
                            i.invoice_id, 
                            i.customer_id, 
                            i.total
                            
                    FROM invoice i            
                    INNER JOIN customer_country cc
                    on cc.customer_id = i.customer_id
                )

SELECT country,
        number_of_customers,
        total_sales,
        average_sales_customer,
        average_order

FROM (
        SELECT  country,
                COUNT(distinct customer_id) number_of_customers, 
                SUM(total) total_sales,
                SUM(total) / COUNT(distinct customer_id) average_sales_customer,
                SUM(total) / COUNT(distinct invoice_id) average_order,
                CASE
                    WHEN country = 'Other' THEN 1
                    ELSE 0
                    END AS sort
            FROM country_total 
            GROUP BY country
        )

ORDER BY sort ASC, total_sales DESC;
        

Done.


country,number_of_customers,total_sales,average_sales_customer,average_order
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143


## Analyzing Full Album Puchases

The Chinook is considering whether they should keep purchasing whole album from records companies.

We will analyze whole album sales and advise the company whether they should keep purchase whole albums instead purchase only the most popular tracks.

In [6]:
%%sql
WITH first_track_invoice AS
                            (
                                SELECT MIN(track_id) first_track_id, invoice_id
                                FROM invoice_line
                                GROUP BY invoice_id
                            )
    
SELECT  album_purchase, 
        COUNT(invoice_id) number_of_invoices,
        CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(invoice_id) FROM invoice) percentage
FROM (
        SELECT fti.*,
            CASE 
                WHEN (SELECT track_id
                      FROM track 
                      WHERE album_id = (SELECT t2.album_id FROM track t2 WHERE t2.track_id = fti.first_track_id)
                      EXCEPT
                      SELECT track_id
                      FROM invoice_line WHERE invoice_id = fti.invoice_id) IS NULL
                AND
                    (SELECT track_id
                     FROM invoice_line WHERE invoice_id = fti.invoice_id
                     EXCEPT 
                     SELECT track_id
                     FROM track
                     WHERE album_id = (SELECT t2.album_id FROM track t2 WHERE t2.track_id = fti.first_track_id)) IS NULL
                THEN 'yes'
                ELSE 'No'
            END AS album_purchase
        FROM first_track_invoice fti
     )
GROUP BY album_purchase;


Done.


album_purchase,number_of_invoices,percentage
No,500,0.8143322475570033
yes,114,0.18566775244299674


Our analysis showed whole album sales comprised almost 20% of the total sales. We will advise the company to keep purchase whole album so they will not lose 20% of the sales.